## Informed Search Algorithms
### Artificial Intelligence 1: week 3

## This week
- Recap: uninformed search algorithms for decision problems

- Heuristic Quality Functions

- General framework for informed search
  - Search guided by cost/quality function  
    Without backtracking:  Hill Climbing (local search)  
    With backtracking: Best-First Search
  - Search taking into account cost of steps taken  
    Dijkstra (path-finding),  A* 
    
- Example applications:
    - inside Machine Learning algorithms
    - path-finding
    - optimisation
    
- Strengths and weaknesses to take into account  
  when selecting a search algorithm to apply to a problem
  

## Recap: common framework
<img src="./figures/generate-and-test-framework.png" style="float:right" >




## Recap: Decision problems:
- yes / no answer (needle in a haystack), so nothing to guide search
- e.g. logic puzzles, combination lock, 
- but lots of real world examples too

Depth-First and Breadth-first search
- up and down the tree or across side to side

In [ ]:
import matplotlib.pyplot as plt
import copy
import numpy as np
from time import sleep
from IPython.display import clear_output
%matplotlib inline




In [ ]:
import notshared
from notshared import DepthFirstSearch
from notshared import BreadthFirstSearch
from notshared import LocalSearch
from notshared import BestFirstSearch
from notshared import AStarSearch
from notshared import DijkstraSearch
from maze import Maze

In [ ]:
from importlib import reload

# import maze
# reload(maze)
# from maze import Maze
# import workbook3_utils
# reload(workbook3_utils)

### Simple function to demonstrate algorithms running on toy maze

In [ ]:
from singlemembersearch import SingleMemberSearch


def test_on_maze(algorithm: SingleMemberSearch):
    mymaze = Maze(mazefile="maze.txt")
    mysearch = algorithm(mymaze, constructive=True, max_attempts=1500)
    found = mysearch.run_search()
    if found:
        print(
            f"search using {mysearch.__str__()} algorithm successful after {mysearch.trials} attempts"
            f" length of path is {len(mysearch.result)} moves."
        )
    else:
        print("solution not found in time allowed")
        print(mysearch.runlog)
    del mymaze

### Depth-First then Breadth-First Search  Behaviour

In [ ]:
test_on_maze(DepthFirstSearch)

In [ ]:
# And now for breadth-first
test_on_maze(BreadthFirstSearch)

## How could we make those better?<img src="figures/timer.png" style="float:right" width = 15%>


Breadth/depth-first generate nodes to test:
-  based on the shape of the tree,
- ignoring  how good the solutions are,  
- or how close they might be to the goal state.

we say they are “blind” or “uninformed”.

More efficient approach is to incorporate information about how close you are to the solution

USE ANYTHING YOU HAVE TO HAND if it helps you avoid constraints!
<img src="figures/multitool.png" style="float:right" width = 20%>

## Quality Functions
Natural for some problems, e.g.:
- **Model Building**: error rate of model on training set,
- **Optimisation**: Distance, cost, payoff
- **Prediction**: error rate of model in real world…

Often more than one
- hence **“heuristic”** (rule of thumb)
- Some may take more effort to calculate
  - simulations run at different fidelity,
  - User studies with different sized groups


## Estimated Quality Measures
For other problems we can define a
	“heuristic evaluation function” , h(n)  for each node n:
 - provides information to **guide**  *informed search*.
 - **estimates** how far a node is from the goal state,

Comparing two nodes m  and n 
- h(m) < h(n) implies m is closer to the goal.
- So typically we look to **minimise** the function.

Also known as a ...  
  cost function  
  quality function,  
  ‘score’,  
  ‘fitness’  (to be maximised)


## Choosing Heuristic Functions
 Should be quick to calculate
 
 Might simplify or ignore constraints (especially 'soft' ones)
 
 The more different levels the better (provide more information to search)
 
 Should be "optimistic" (underestimate distance/cost)
  - e.g. training set accuracy underestimates error on unseen data

### Example: Two landscapes created from the same function
### with quality calculated at different levels of precision
![integer vs 1 decimal place](./figures/2landscapes.png)

## Adding heuristic functions to our generate and test code

Minor change to pseudocode we had for depth and breadth first:

Now our Evaluate() function gives some idea of quality instead of just feasibility

So all we need to change is the method  ```SelectAndMoveFromOpenList(algorithm_name)```

Typically to return the one with the highest quality
 - Could do the same with uninformed search using quality = age
 - but by using a list,     
    and always adding new things to the end,  
    we get that for free

The function may also do some memory management of the open list.
- **next year in AI2 you'll see how extending this gives rise to some more complicated search algorithms**

## Pseudocode for informed search -  same as before changing the line in pink
<div style="background:#F0FFFF;font-size:18pt">
<p style="color:darkred;font-size:18pt;margin-bottom:0pt"><em>MAIN LOOP</em></p>
<dl style="font-size:18pt;margin-top:0pt">
    <dt>&nbsp;&nbsp;&nbsp;<b>WHILE</b> IsNotEmpty( open_list) <b>DO</b> </dt>
    <dd> <span style="background:pink">working_candidate &larr; <b>SelectAndMoveFromOpenList</b>(algorithm_name)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:red"> <b>Algorithm</b>-specific code</span></span></dd>
    <dd>  <b>FOR</b> sample in SAMPLE_SIZE <b>DO</b> <br>
        <dl style="font-size:18pt">
            <dt style="color:blue;font-style:italic"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GENERATE </dt>
            <dd>  neighbour &larr; <b>ApplyMoveOperator</b> (working_candidate)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:red"> <b>Representation</b>-specific code</span></dd>
            <dt style="color:blue;font-style:italic">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;TEST</dt>
            <dd> status &larr; <b>Test</b> ( neighbour)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:red"> <b>Problem</b>-specific code</span></dd>
            <dt style="color:blue;font-style:italic"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;UPDATE WORKING MEMORY</dt> 
            <dd> <b>IF</b> status IS AtGoal <b>THEN</b><br>
                    &nbsp;&nbsp;&nbsp; <b>Return</b>(SUCCESS)</dd>
            <dd> <b>ELSE IF</b> status IS BREAKS_CONSTRAINTS <b>THEN</b><br>
                &nbsp;&nbsp;&nbsp; <b>AppendToClosedList</b>(neighbour)</dd>
            <dd><b>ELSE</b><br>
                &nbsp;&nbsp;&nbsp; <b>AppendToOpenList</b>(neighbour)</dd>
        </dl>
    <dd>          <b>AppendToClosedList</b>(workingCandidate)</dd>
</dl>
</div>    

## Simple example: Hill Climbing local search

Always accept first move i
Sorts list by decreasing quality <=> increasing distance_to_goal

**Doesn’t allow back-tracking **
  - Discard all but first node after sorting.

Conceptually: examine child-nodes then:
 - move to one with better heuristic value if exists,
 - else stop (even if goal / global optimum not reached)

greedy/steepest ascent variants:
 - Examine all child nodes and follow first / best  improvement.

**Quick but gets stuck in local optima**


In [ ]:
reload(notshared)
from notshared import LocalSearch

In [ ]:
test_on_maze(LocalSearch)

## Hill Climbing can get stuck even on our simple example! <img src="figures/hillclimbing-tree.png" style="float:right" width=50%>

- But is fast
- and you can always restart it from another place

Examples:
- Optimisation:
  - timetabling
- Machine Learning
  - most decision tree algorirthms
   - greedy rule induction     
- "Stochastic Gradient Descent" (aka backprop)
  for training neural networks


## Best First Search <img src="figures/best-first-tree.png" style="float:right" width = 50%>
Like hill-climbing is driven by quality  
**but keeps unused nodes in the open list**

At every iteration:
- sort **whole queue** by decreasing quality,  
  instead of just sorting children of current node.
- i.e. doesn’t removed unexplored nodes  
  This adds  backtracking

Tends to produce shorter paths than depth- or breadth first search


### pseudocode for best-first search (main loop only)
    WHILE ( Openlist not empty) DO
      SORT(OpenList by increasing distance to goal)  ## quality
      MOVE (first item from openList into working candidate)
      FOREACH (1-step neighbour)
        neighbour = ApplyMoveOperator(workingCandidate)  
        Evaluate(neighbour)                               
	    IF(IsAtGoal(neighbour))
          OUTPUT (SUCCESS, neighbour)
        ELSE IF (neighbour is feasible) ##don't stop at plateau 
          APPEND( neighbour to end of openList)
        ELSE
          APPEND( neighbour to end of closedList)
      COPY (working candidate to closedList)



<div class="alert alert-block alert-danger"> note best-first get back-tracking by omitting the line <b>DELETE(all but first item in openlist)</b>  </div> 

In [ ]:
reload(notshared)
from notshared import BestFirstSearch

In [ ]:
test_on_maze(BestFirstSearch)

## Quiz Questions:
- Hill-Climbers can get stuck in local optima (True:False)
- The local optima a hill-climber finds, depends on where it starts (True:False)
- Which of these might help local search?
Vote for as many as you think will
  - Multiple runs from random starting places
  - Multiple runs, start each one by making random changes to  the last local optimum
  - One  run, changing the move operator everytime you reach a local optimum
  - Sometimes accepting worsening moves


# Pause

## Taking into account the cost of reaching a solution
 <img src="figures/search/balanced_plates.jpg" style="float:right">


E.g. planning 
- Routes to avoid toll roads (cost)  
- Routes to avoid  built-up areas (air pollution)
- the path of a manipulator to reduce number of moves
- the path of a manipulator avoiding sudden changes of direction  



## A* : **guaranteed** shortest/least cost paths <img src="figures/optimal.png" style="float:right" width = 25%>
Adds cost to Best-First to find optima
 - Shortest path / least complex model,

Sorts the list of unexplored nodes by f(node):
- f(node) = g(node) + h(node),  
  h(node) =  estimated distance to goal (heuristic).  
  g(node) = cost of reaching that node.

So you can stop looking as soon as  you know that   
g(node) > best_so_far for all remaining nodes

**A* is complete and optimal as long as h(node) is an underestimate**

A* is used for : path-finding (especially NPCs in games),   query optimisation, …


## pseudocode for A* (main loop only)

    WHILE ( Openlist not empty) DO
      SORT(OpenList by combined distance to goal and cost)  ## quality
      MOVE (first item from openList into working candidate)
      FOREACH (1-step neighbour)
        neighbour = ApplyMoveOperator(workingCandidate)  
        Evaluate(neighbour)                               
        IF(IsAtGoal(neighbour))
          OUTPUT (SUCCESS, neighbour)
        ELSE IF (neighbour is feasible) ##don't stop at plateau 
          APPEND( neighbour to end of openList)
        ELSE
          APPEND( neighbour to end of closedList)
      COPY (working candidate to closedList)
<div class="alert alert-block alert-danger"> This is just best-first with a modified sort condition.<br> You could modify the IF statement to only add things to openlist if g(neighbour) < f(workingCandidate)  </div> 

In [ ]:
reload(notshared)
from notshared import AStarSearch

In [ ]:
test_on_maze(AStarSearch)

## A* example <img src="figures/Astar-tree.png" style = "float:right" width = 40%>
 We show the cost as a second number in each node – in this case just the depth

## What does "optimality" mean for A* ?
Finds node which satisfies the goal criteria

If there is more than one of these, it finds the one with the least cost 

How else could we interpret this?
What might be desirable?




# Dijktra's algorithm
<img src = "figures/dijkstra.gif" style = "float:right" width = 40%>

Designed for use in tracing routes between points in an weighted undirected graph.
- "weighted" means there are cost/distances on each link   
  (edge in the graph)  
  e.g. tolls/ different terrains, ...
- "undirected" means you can traverse an edge  
  in either direction ( no one-way roads)

Finds the single shortest path between two points.  
Most used for path-finding – a *lot* in games

Like A* but ignores heuristic cost
h(n) = 0 for all n

"Dijkstra Animation" by Ibmua - Work by uploader.. Licensed under Public Domain via Commons - https://commons.wikimedia.org/wiki/File:Dijkstra_Animation.gif#/media/File:Dijkstra_Animation.gif


## What does this mean in terms of our existing pseudocode?
Solution = node in graph = location on grid
- Representation:
  - 2 ints for {x,y} co-ordinates
  - (optional) Add parent location to trace route afterwards
- Distance from start
- Could add distance to goal = but ignored by algorithm so why bother?

Sort criteria for picking working candidate from open list = closest distance to start

## pseudocode for Dijkstra (main loop only)


    WHILE ( Openlist not empty) DO
      SORT(OpenList by increasing distance from start)  ## distance travelled
      MOVE (first item from openList into working candidate)
      FOREACH (1-step neighbour)    neighbour = ApplyMoveOperator(workingCandidate)  
        Evaluate(neighbour) ##using route through workingCandidate                               
        IF(IsAtGoal(neighbour))
           OUTPUT (SUCCESS, neighbour)
        ELSE IF (neighbor is already on openList) ## might have new
           UPDATE(neighbor.dist_to_start on openList) ##quicker route to neighour
        ELSE IF(neighbor is feasible)
           ADD( neighbor to end of openList)
        ELSE
           ADD( neighbor to end of closedList)
      COPY (working candidate to closedList)    

<div class="alert alert-block alert-danger"> This is just best-first with a modified sort condition and two extra lines:<br>
           ELSE IF (neighbor is already on openList) ## might have new <br>
           UPDATE(neighbor.dist_to_start on openList) ##quicker route to neighour
</div> 

In [ ]:
reload(notshared)
from notshared import DijkstraSearch

In [ ]:
test_on_maze(DijkstraSearch)

## Really good description and discussion of how different algorithms can be used for path-finding

http://www.redblobgames.com/pathfinding/a-star/introduction.html
    
    

## Quiz Questions
For a ‘decision’ problem, which of these   would be appropriate ?

For an exam timetabling problem which of these   would be appropriate ?

For a npc planning a path to chase someone in a game, which of these   would be appropriate ?

For organising daily delivery schedules, which of these   would be appropriate ?

- Depth-first
- Breadth-first
- Hill-Climbing
- Best-First
- A*
- Dijkstra


## Strengths and weaknesses of different approaches

You should construct a tables listing whether different algorithms possess these features

- Completeness?
  - will get stuck in local optima?
  - avoids getting stuck in loops by design?
  - can (sometimes) be adapted to avoid being stuck in loops? 
- Efficiency, e.g.
  - Can make more efficient by incorporating information about solution quality?
  - Limited storage overheads
- Optimality:
  - Guaranteed to find a solution if one exists?
  - Can find least-cost/short/least complex solutions?
  
 And look at the reading lists, examples of applications using search algorithms  (sat navs, machine learning, ...) to understand why they make the choices and trade-offs they do

## Summary of search topic:
You need to know about and recognise:
- Common framework
- Depth and Best first search when there is no quality function
- Characteristics of a good heuristic quality function
- Simple Hill Climber
- Best first
- A*
- Dijkstra’s Algorithm

You should be able to answer questions about
- How to implement different strategies within a common framework
- Choosing an appropriate search strategy for a problem:
  - Do you have a way of assigning quality?
  - What are your trade-offs for time vs storage vs optimality?
  - Can ‘good-enough’ be ok?


# Extras: Optimising TSP

In [ ]:
import random, numpy as np, math, matplotlib.pyplot as plt

# place cities in random positions


def get_distances(cities):
    num_cities = len(cities)
    distances = np.zeros((num_cities, num_cities))
    for row in range(num_cities):
        for col in range(num_cities):
            if row != col:
                xdist = cities[row][0] - cities[col][0]
                ydist = cities[row][1] - cities[col][1]
                distances[row][col] = math.sqrt(xdist * xdist + ydist * ydist)
    return distances


def plot_cities(cities):
    fig, ax = plt.subplots()
    for i in range(len(cities)):
        ax.plot(cities[i][0], cities[i][1], "Xb")
    modelstrings = np.array(["%.2f" % x for x in model.reshape(model.size)])
    modelstrings = modelstrings.reshape(model.shape)
    ax.table(cellText=modelstrings, loc="right", bbox=[1.1, 0, 1, 1])
    plt.show()


def show_tour(cities, start, tour):
    num_cities = len(cities)
    plt.plot(cities[start][0], cities[start][1], "or", markersize=12)
    plt.plot(
        [cities[tour[i % num_cities]][0] for i in range(num_cities + 1)],
        [cities[tour[i % num_cities]][1] for i in range(num_cities + 1)],
        "Xb-",
    )
    plt.show()

In [ ]:
num_cities = 10
cities = [random.sample(range(100), 2) for x in range(num_cities)]
model = get_distances(cities)
plot_cities(cities)

In [ ]:
def greedyConstructive(cities, start=0):
    num_cities = len(cities)
    tour = [-1 for i in range(num_cities)]
    tour[start] = 0
    # loop through stops on the tour
    for i in range(1, num_cities):
        min_dist = 100000
        # finding the next closest unvisited place
        for k in range(num_cities):
            dist_ik = model[tour[i - 1]][k]
            if tour[k] == -1 and dist_ik < min_dist:
                min_dist = dist_ik
                min_k = k
        tour[min_k] = i
    return tour


start = random.randint(0, num_cities - 1)
tour = greedyConstructive(cities, start)
print(tour)
show_tour(cities, start, tour)

In [ ]:
def getNextItemForAlgorithm(algorithm, openList):
    next = -1
    numEntries = len(openList)
    # check openList is not empty
    if numEntries == 0:
        print("openList was empty!")

    else:
        if algorithm == "depthFirst":
            # return last thing added
            next = len(openList) - 1

        elif algorithm == "breadthFirst":
            # return oldest thing on list
            next = 0

        elif algorithm == "bestFirst" or algorithm == "localSearch":
            # loop through list looking for entry with highest quality
            best = 0
            for pos in range(1, (numEntries - 1)):
                if openList[pos].quality < openList[best].quality:
                    best = pos
            next = best

        elif algorithm == "Astar":
            # loop through list lookinmg for item with lowest sum of estimated distance ot goal + number of moves from start
            best = 0
            for pos in range(1, (numEntries - 1)):
                if openList[pos].quality + len(openList[pos].variableValues) < (
                    openList[best].quality + len(openList[best].variableValues)
                ):
                    best = pos
            next = best

        else:
            print("unrecognised algorithm")

    return next

In [ ]:
#### INITIALISE SEARCH ###
num_cities = 10
cities = [random.sample(range(100), 2) for x in range(num_cities)]
model = get_distances(cities)
plot_cities(cities)


# change initialise os that it takes a model of distances rathe than a maze
def initialise(model):
    num_cities = model.shape[0]
    workingCandidate = CandidateSolution()
    # get start position  and set this as start for search
    start = random.randint(0, num_cities - 1)
    workingCandidate.variableValues.append(start)

    # measure quality
    workingCandidate.quality = evaluate(workingCandidate, model)

    # check for lucky guess
    if IsAtGoal(workingCandidate, maze):
        print("solution found")
        atGoal = True
    else:
        openList = []
        closedList = []
        openList.append(workingCandidate)
        atGoal = False

    return workingCandidate, openList, closedList, atGoal